# Algoritmo: Agrupamento perfis de velocidades
## Autor: Antônio Claudio Dutra Batista
## Orientador: Francisco Moraes de Oliveria Neto
## Metodologia: Agupamento de perfis de velocidades com consideração do tempo
''' Os dados são de velocidade obtidas pela extração de informações do sistema de localização geografrica de veiculos em operação em um trecho de estudo e para tal situação se deseja identificar os perfis de velocidades veicular semelhantes '''

# 1.0 Preparando dados para manipulação

In [ ]:
# Importando bibliotecas necessárias
import imgkit
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
import seaborn as sns
import plotly.graph_objs as go
import plotly.io as pio

In [ ]:
# Trazendo base com dado de velocidades dos veiculos no trecho em estudo
df1 = pd.read_excel('velocidades_discretizadas_test.xlsx')

In [ ]:
df1

In [ ]:
# Retribuindo variavel para manipulação e dropando coluna distancia 
df_concatenado = df1.drop(columns='Distância (m)')

In [ ]:
# Transpondo o DataFrame para que as colunas representem instâncias
df_transposto = df_concatenado.T

In [ ]:
# Normalização das velocidades (Não foi normalizado)
scaler = StandardScaler()
df_normalizado = df_transposto # scaler.fit_transform(df_transposto)

In [ ]:
df_normalizado

## 1.1 Correlação e descrição entre as colunas

In [ ]:
# DataFrame
df = df_concatenado

# Calculando a matriz de correlação
correlation_matrix = df.corr()

# Plotando um mapa de calor
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.show()

In [ ]:
# Obtendo estatísticas descritivas
df.describe()

## 1.2 Método do cotovelo (Elbow Method)
''' Será utilizada essa técnica para encontrar o número ideal de clusters para aplicação do algoritmo de agrupamento! '''

In [ ]:
# Lista para armazenar as distorções (soma dos quadrados intra-cluster) para cada número de clusters
distortions = []

# Número máximo de clusters que você deseja testar
max_clusters = 10 

# Iterando sobre diferentes números de clusters e calculando as distorções
for i in range(1, max_clusters + 1):
    kmeans_colunas = KMeans(n_clusters=i, random_state=42)
    kmeans_colunas.fit(df_transposto)
    distortions.append(kmeans_colunas.inertia_)

In [ ]:
# Plotando o gráfico do método do cotovelo
plt.plot(range(1, max_clusters + 1), distortions, marker='o')
plt.title('Método do Cotovelo')
plt.xlabel('Número de Clusters')
plt.ylabel('Distorção (Soma dos Quadrados Intra-Cluster)')
plt.show()

In [ ]:
''' O ponto ideal é onde ocorre uma distorção onde (inertia) começa a diminuir mais lentamente "Que forma um cotovelo" no gráfico'''

# 2.0 Aplicação do algoritimo de agrupamento (k-means)

In [ ]:
# Importando da biblioteca funções da biblioteca necessária
from sklearn.preprocessing import StandardScaler
import threadpoolctl

In [ ]:
# Numero de clusters (k)
k = 4 # Parametro de numeros de clusters que é necessário determinar o mais adequado!

In [ ]:
''' Olhar no grafico do Método do cotovelo o k mais ajustado! '''

In [ ]:
# Aplicando o algoritimo (k-means)
kmeans = KMeans(n_clusters=k, random_state=42)
labels_colunas = kmeans_colunas.fit_predict(df_transposto)

In [ ]:
# Adicionando rótulos de cluster aos dados originais
df_transposto['cluster'] = labels_colunas

In [ ]:
df_resultado = df_transposto.T

In [ ]:
# Exibindo reultado 
df_resultado

In [ ]:
df_resultado.tail(25)

In [ ]:
# Verificando a quantidade de CLUSTERSabs
ultima_linha = df_resultado.iloc[-1]
ultima_linha.unique()

In [ ]:
''' ÚLTIMA LINHA SÃO OS CLUSTERS IDENTIFICADOS! '''

## 2.1 Visualização dos clusters identificados
''' Selecioando cluster especifico '''

In [ ]:
# FILTRANDO CLUSTERS QUE SE DESEJA ANALISAR
colunas_filtradas = []
for coluna in df_resultado.columns:
    if df_resultado[coluna].iloc[-1] == 4: # filtrar aqui!
        colunas_filtradas.append(coluna)

# Filtrando o DataFrame mantendo apenas as colunas filtradas
df_filtrado = df_resultado[colunas_filtradas]

In [ ]:
df_filtrado # Base espefica aggrupada

In [ ]:
df_filtrado.describe() # Descrição dos perfis semelhantes filtrados

In [ ]:
# Calculando a matriz de correlação PARA DADOS AGRUPADOS
correlation_matrix = df_filtrado.corr()

# Plotando um mapa de calor
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.show()

In [ ]:
# Visualização dos clusters
# Removendo a última linha que contém os rótulos de cluster para visualização
df_resultado_sem_clusters = df_filtrado.iloc[:-1]

# Obtendo os rótulos de cluster da última linha
rótulos_cluster = df_filtrado.iloc[-1, :]

plt.figure(figsize=(8, 7))
# Iterando sobre cada coluna, plotando e identificando pelo número do cluster
for coluna in df_resultado_sem_clusters.columns:
    plt.scatter(df_resultado_sem_clusters.index, df_resultado_sem_clusters[coluna], label=f'Coluna {coluna} - Cluster {rótulos_cluster[coluna]}')

# Ajusta o tamanho da fonte da legenda
plt.legend(fontsize='small')

# Convertendo os índices para um tipo numérico
x = df_resultado_sem_clusters.index.astype(float)
y = df_resultado_sem_clusters.mean(axis=1)

# Adicionando uma linha de tendência
z = np.polyfit(x, y, 3)  # ajuste polinomial de 3º grau
p = np.poly1d(z)
plt.plot(x, p(x), 'r--', label='Linha de Tendência')

plt.title('Clusters por Coluna', fontsize='xx-large')
plt.xlabel('Índice')
plt.ylabel('Velocidades')
plt.show()

## 2.1.2 Visualização perfis veiculares do mesmo clusters

In [ ]:
# Importando base com as distancias entre estaçoes (df2 é a base com distania entre paradas!)
df2 = pd.read_excel('PARADAS_interesse_tudo_com_distancias (1).xlsx')

In [ ]:
# BASE COM DISTANIAS: 
# df2

In [ ]:
# Defindo o valor da primeira linha de uma coluna distancia acumulado como zero (referencia)
df2.loc[0, 'distancia_acumulada_paradas'] = 0

In [ ]:
# Descartando a última linha do DataFrame (contem os clusters)
df_sem_ultima_linha = df_filtrado.drop(df.index[-1])

# Adicionado coluna de distancia 
df_sem_ultima_linha['Distância (m)'] = df1['Distância (m)']

In [ ]:
df_sem_ultima_linha

In [ ]:
# Criando traces para as colunas de velocidade
traces = [
    go.Scatter(
        x=df_sem_ultima_linha['Distância (m)'],
        y=df_sem_ultima_linha[coluna],
        mode='lines',
        name=coluna
    )
    for coluna in df_sem_ultima_linha.columns[:-1]  # Excluindo a última coluna, que é a coluna de distância
]

# Adicionando marcadores da base de dados df2
trace_df2 = go.Scatter(
    x=df2['distancia_acumulada_paradas'],
    y=np.zeros(len(df2)),  # Todos os marcadores estarão na linha y = 0
    mode='markers',
    marker=dict(symbol='triangle-up', size=10, color='black'),  # Define os marcadores como triangulares
    hovertext=df2['stop_name'],  # Define o texto que aparecerá ao passar o mouse sobre os marcadores
    hoverinfo='text',
    showlegend=False  # Não mostrar esse trace na legenda
)
traces.append(trace_df2)

# Calculando os percentis 5 e 95 das velocidades
velocidades = df_sem_ultima_linha.drop(columns='Distância (m)').values.flatten()
percentil_5 = np.percentile(velocidades, 5)
percentil_95 = np.percentile(velocidades, 95)

# Criando o layout
layout = go.Layout(
    title='Perfis de Velocidades',
    xaxis=dict(title='Distância (m)'),
    yaxis=dict(title='Velocidade (km/h)', range=[-5, percentil_95]),  # Definindo o range do eixo y
    hovermode='closest'
)

# Criando a figura
fig = go.Figure(data=traces, layout=layout)

# Plotando o gráfico
fig.show()

In [ ]:
# Salvar o gráfico como um arquivo HTML
pio.write_html(fig, file='perfil_de_velocidade.html')

In [ ]:
''' OBS: NA OPAÇÃO DE CAMERA NA IMAGEM DO PERFIL PE POSSIVEL BAIXAR O PNG DA IMAGEM! '''

# ---------------------------------------------------------------------